In [1]:
### 패키지 설치

import pandas as pd #Analysis 
import matplotlib.pyplot as plt #Visulization
import seaborn as sns #Visulization
import numpy as np #Analysis 
from scipy.stats import norm #Analysis 
from sklearn.preprocessing import StandardScaler #Analysis 
from scipy import stats #Analysis 
import warnings 
warnings.filterwarnings('ignore')
%matplotlib inline
import gc #garbage collector

import os
import string
color = sns.color_palette()

%matplotlib inline

from plotly import tools #웹 시각화 라이브러리인 d3.js 를 이용하여 보다 interactive 하게 그래프를 만들어주는 모듈.
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

from sklearn import model_selection, preprocessing, metrics, ensemble, naive_bayes, linear_model
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer #TfidVectorizer: Term Frequency-Inverse Document Frequency(단어 빈도- 역문서 빈도)
from sklearn.decomposition import TruncatedSVD #특이값 분해(Singular Value Decomposition, SVD)

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999

import plotly.graph_objs as go

import time
import random

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

In [3]:
from lightgbm import LGBMRegressor

ELI5 is a Python library which allows to visualize and debug various Machine Learning models using unified API. It has built-in support for several ML frameworks and provides a way to explain black-box models.

In [4]:
### Permutation importance
import eli5 
from eli5.sklearn import PermutationImportance

SHAP 각 예측과 변수에 대한 같은 질문을 함으로써 모델의 설명을 build 한다.

"변수 j가 이 모델로부터 제거될 때 얼마나 이 예측 i 에 변화를 줄까?"

그래서 이 물음에 대한 답은 SHAP values로 표현된다.

SHAP values 는 한 예측에서 변수의 영향도를 방향과 크기로 표현한다.

In [5]:
###shap
import shap 
#SHapley Additive exPlanations

목차
데이터 탐색  
결측치 제거 및 유니크 값  
시간에 따른 탐색  
Amount의 심화 탐색  
installments(할부) 심화 탐색  
클러스터링  
일별 거래의 변화율  
시간별 거래의 변화율  
Prophet을 통한 클러스터 별 트렌드 분석  
변수중요도  
Feature Engineering  
Correlation  
Tree Importance  
Permutation Importance  
Shap value  

In [6]:
train = pd.read_csv("train.csv",parse_dates=['date'])
train.shape

(3362796, 8)

"parse_dates의 기능 : CSV 파일에서 날짜 텍스트 파일을 감지하면 datetime64[ns]로 지정"  
dayfirst=True 일이 월보다 먼저 위치하는 것으로 설정  
infer_datetime_format=True 날짜시간 포맷 추정해서 파싱하기  

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3362796 entries, 0 to 3362795
Data columns (total 8 columns):
 #   Column        Dtype         
---  ------        -----         
 0   store_id      int64         
 1   date          datetime64[ns]
 2   time          object        
 3   card_id       object        
 4   amount        int64         
 5   installments  float64       
 6   days_of_week  int64         
 7   holyday       int64         
dtypes: datetime64[ns](1), float64(1), int64(4), object(2)
memory usage: 205.2+ MB


변수는 총 8개이고, 관측치는 330만개로 관측치에 비해 변수가 많이 적은 상황. 그리고 각 변수의 설명은 아래와 같음.  

store_id :각 파일에서의 상점 고유 번호.  
date : 거래 일자  
time :거래 시간  
card_id :카드 번호의 hash 값  
amount :매출액  
installments : 할부개월수. 일시불은 빈 문자열  
days_of_week : 요일, 월요일이 0, 일요일은 6  
holiday : 1이면 공휴일, 0이면 공휴일이 아님  

In [8]:
train.head()

,store_id,date,time,card_id,amount,installments,days_of_week,holyday
0,0,2016-12-14,18:05:31,d297bba73f,5,NaN,2,0
1,0,2016-12-14,18:05:54,d297bba73f,-5,NaN,2,0
2,0,2016-12-19,12:42:31,0880849c05,144,NaN,0,0
3,0,2016-12-19,12:48:08,8b4f9e0e95,66,NaN,0,0
4,0,2016-12-19,13:31:08,7ad237eed0,24,NaN,0,0


데이터는 크게 3가지로 구분되어 짐. 시간, 식별화 코드, 매출  

시간 : date, time, days_of_week, holiday  
식별화 코드 : store_id, card_id  
매출 : amount, installments  
ㅡ> 참고 : 변수명 중에 holyday는 holiday가 오타난것 같음  
 
아이디어  

row 1을 보면 amount가 음수인게 있는데 이는 취소된 거래를 말하는 것임. 바로 위 row 0의 거래가 3초만에 취소되었다는것을 의미.  
date와 time의 경우 좀 더 구분의 편의를 위해서 년도/월/일/시간/분으로 변수를 확장 함.  
days_of_week도 월~금, 토 ~ 일요일인지 여부를 나누어서 비교.  
installments에 보면 결측치가 있는데, 이는 일시불을 의미하는것이므로 1으로 처리.  

In [9]:
def mis_value_graph(data):  
    data = [
    go.Bar(
        x = data.columns,
        y = data.isnull().sum(),
        name = 'Counts of Missing value',
        textfont=dict(size=20),
        marker=dict(
        line=dict(
            color= generate_color(),
            #width= 2,
        ), opacity = 0.45
    )
    ),
    ]
    layout= go.Layout(
        title= '"Total Missing Value By Column"',
        xaxis= dict(title='Columns', ticklen=5, zeroline=False, gridwidth=2),
        yaxis= dict(title='Value Count', ticklen=5, gridwidth=2),
        showlegend=True
    )
    fig = go.Figure(data=data, layout=layout)
    py.iplot(fig, filename='skin')
    
def generate_color():
    color = '#{:02x}{:02x}{:02x}'.format(*map(lambda x: random.randint(0, 255), range(3)))
    return color

mis_value_graph(train)

ploty에서 막대그래프를 만드는 함수 go.Bar opacity는 불투명도